# iLOrest examples


Version 0.33

## Introduction

This Jupyter Notebook contains iLOrest examples as well as tips and tricks answering recurrent questions from HPE customers.

## List of examples

Note that the list of examples present in this Jupyter Notebook is changing overtime. The classification is not following any strict model; it is a mix of Root service and data type locations.

* AccountService
   - [iLO users: Enforce password complexity and minimum password length](#EnforceiLOUsersPasswordComplexity)

* Chassis
   - [ThermalConfiguration](#ThermalConfiguration)
    
* Managers (iLO)
    - [Enable/Disable SNMPv1 in iLO](#SNMPv1Enabled)
 
* Resource Directory
    - TBD

* CompusterSystem
     - [PostState and DiscoveryComplete](#PostState_And_DiscoveryComplete)
     - [Bios/RBSU Login password](#ManageBiosRbsuPassword)

## Environment preparation

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation ((Version: 0.1)) ############


# Set Student ID number
Stud={{ STDID }}

# location and ports variables
let OpenBmcPort=44000+${Stud}
let iLO5SimulatorPort=45000+${Stud}
let ilo5Port=443

CacheDir="${PWD}/iLOrestCache"
LogDir="${PWD}"
LogFile="${LogDir}/iLOrest.log"

iLO5SimulatorIP=bmcsimulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

OpenBmcIP=bmcsimulators
OpenBmc="${OpenBmcIP}:${OpenBmcPort}"
OpenBmcURI="https://${OpenBmc}"

ilo5IP="ilo5"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password='P@ssw0rd!'

# Miscellaneous
alias ilorest='ilorest --nologo --cache-dir ${CacheDir} --logdir ${LogDir}'
ilorest logout &>/dev/null # Clear cache


# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

### iLO 5 login and cache redirection

This infrastructure contains only one physical computer and prevents multiple students to perform SET actions on this unique system. Therefore, students have to use a DMTF BMC simulator populated with an iLO 5 mockup. However, because iLOrest populates its cache directory during the login process you need first to authenticate with the iLO 5 and then modify the cache files to have them pointing to the DMTF simulator. 

In [ ]:
ilorest login $ilo5 -u $User -p $Password

# Copy cache in a safe location before logging out to
# minimize opened sessions. iLO 5 is limited to 15 concurrent
# sessions. 
mv ${CacheDir}/cache{,-bck}
ilorest logout

# Move back the cache
mv ${CacheDir}/cache{-bck,}

# Redirect cache toward the iLO 5 Simulator
sed -i "s.${ilo5URI}.${iLO5SimulatorURI}." ${CacheDir}/cache/*

## AccountService

This root service models the iLO User Accounts.

<a id = "EnforceiLOUsersPasswordComplexity">iLO users: Enforce password complexity and minimum password length</a>\
API Reference document keyword: `EnforcePasswordComplexity` \
iLO GUI location: `Security - Access Settings`

In [ ]:
# Select DMTF data type 
ilorest select AccountService

# List properties of the HPE Oem extension in json format 
ilorest list Oem/Hpe --json

# Modify two AccountService parameters
ilorest set Oem/Hpe/EnforcePasswordComplexity=True
ilorest set Oem/Hpe/MinPasswordLength=5

# Retrieve the list of modified paramters
ilorest status

# Uncommnent the following comand to commit changes in the iLO 5 simulator.
# Note that you will not get the same answer as if done on a real iLO 5.
#ilorest commit

# Undo changes
ilorest select AccountService --refresh

## Chassis

This section contains parameters related to the `Chassis`data type as well as data types under the Oem.

### <a id = "SerialNumber">SerialNumber</a>

[API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#serialnumber294) keyword: `SerialNumber` \
iLO GUI location: `Information - iLO Overview` 

> **NOTE** The server

In [ ]:
# Select data type
ilorest select Chassis

# Get SerialNumber from the Chassis data type
ilorest get SerialNumber

# Note that this property is READ-ONLY
ilorest info SerialNumber

### <a id = "ThermalConfiguration">ThermalConfiguration</a>

> **NOTE** This parameter is new with iLO 5 firmware 2.30. 

    API Reference document keyword: `TBD: Not yet documented` \
    iLO GUI location: `TBD: Not yet implemented`

In [ ]:
# Select data type
ilorest select HpeThermalExt
 
#ilorest list --json

## Managers

Examples in this section are related to the `Manager` data type as well as data Oem data types located below `/redfish/v1/Managers/` end point like `HpeiLOSnmpService`. 

### <a id = "SNMPv1Enabled">Enable/Disable SNMPv1 in iLO</a>

> **NOTE** Changing the `SNMPv1Enabled` setting appeared in iLO 5 FW 2.30.

    API Reference document keyword: `SNMPv1Enabled` \
    iLO GUI location: `TBD`

In [ ]:
# Select data type
ilorest select HpeiLOSnmpService

# Get current value
ilorest get SNMPv1Enabled
echo

# Set new value
ilorest set SNMPv1Enabled=False

ilorest status

## HPE iLO Resource Directory

The HPE iLO Resource Directory is an HPE Oem extention containing the exhaustive list of the data types present the in a Redfish pecific data types. This directory can be used by Redfish clients to find faster find a specific resource in the Redfish tree.

In [ ]:
Type="HpeThermalExt"
#Type="SoftwareInventory"
ilorest types | grep Resource
ilorest select HpeiLOResourceDirectory
echo
ilorest list instances --json | jq -r '.[]|.[]|."@odata.type"' \
         | awk -F. '/'${Type}'/ {print $1}' | sort -u

## ComputerSystem

This section contains examples related to the `ComputerSystem` data type.

### <a id = "PostState_And_DiscoveryComplete">PostState and DiscoveryComplete</a>

Use these state parameters to evaluate the current state of a system before modifying or viewing its resources. As explained in this [article](https://developer.hpe.com/blog/master-the-redfish-server-states-to-improve-your-monitoring-and-manageme) certain parameters cannot be modified if the computer is not in a specific state.

Moreover, some parameters cannot be viewed when the entire discovery of all subsystems is not complete. As an example, you will not be able to view or modify SmartArray and logical drive configurations if the `SmartArrayDiscovery` is not complete.

In [ ]:
ilorest select ComputerSystem
# Try to include HpeComputerSystemExt in the Mockup 
ilorest get Oem/Hpe/DeviceDiscoveryComplete
echo
ilorest get Oem/Hpe/PostState 
echo 


### <a id = "ManageBiosRbsuPassword">Bios/RBSU Login password</a>

> NOTE: requires iLOrest >= 3.1.1

In [ ]:
# Setting a password for the first time
ilorest setpassword FirstTimePassword '""'

# Removing the Bios/RBSU password
ilorest setpassword '""' FirstTimePassowrd

# No commit needed. No iLO nor system reset needed. 
ilorest status


## Thermal

In [ ]:
ilorest serverinfo --thermal
ilorest list --json

### Serial Number

from BIO datatype

In [ ]:
# Select data type
ilorest select Bios.

# Get SerialNumber from Bios data type.
ilorest get Attributes/SerialNumber

# Get more info on s
ilorest info Attributes/SerialNumber --latestschema

echo "Set SN"
ilorest set Attributes/SerialNumber="CZJ93402YC"

echo " status ?"
ilorest status --latestschema

In [ ]:
ilorest logout

## TBD
